In [ ]:
#import os
#os.getcwd()

In [83]:
#!pip install pytorch_lightning
#!pip install pytorch_forecasting

In [84]:
import sys
import os
import argparse
import shutil
import random
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import pytorch_lightning as pl

from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
)
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    LearningRateMonitor
)
from pytorch_forecasting.metrics import SMAPE
from pytorch_forecasting.models import TemporalFusionTransformer


# category columns
CATE_COLS = ['num', "mgrp", 'holiday', 'dow', 'cluster', 'hot', 'nelec_cool_flag', 'solar_flag']

# building cluster based on kmeans
CLUSTER = {
    0: [19, 20, 21, 49, 50, 51],
    1: [1, 5, 9, 34],
    2: [4, 10, 11, 12, 28, 29, 30, 36, 40, 41, 42, 59, 60],
    3: [2, 3, 6, 7, 8, 13, 14, 15, 16, 17, 18, 22, 23, 24, 25, 26, 27, 31, 32, 33, 35, 37, 38, 39, 43, 44, 45, 46, 47, 48, 52, 53, 54, 55, 56, 57, 58],
}

# length of training data for prediction (5 weeks)
ENCODER_LENGTH_IN_WEEKS = 5

# learning rate determined by a cv run with train data less 1 trailing week as validation 
LRS = [0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306 , 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307,
       0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307]

# number of epochs found in cv run
NUM_EPOCHS = 66

# number of seeds to use
NUM_SEEDS = 10

BATCH_SIZE = 128

# hyper parameters determined by cv runs with train data less 1 trailing week as validation 
PARAMS = {
    'gradient_clip_val': 0.9658579636307634,
    'hidden_size': 180,
    'dropout': 0.19610151695402608,
    'hidden_continuous_size': 90,
    'attention_head_size': 4,
    'learning_rate': 0.08
}

In [ ]:
#os.getcwd()

In [86]:
#경로만 자신의 환경에 맞게 잘 설정해주세요!
DATAROOT='/data'
CKPTROOT = DATAROOT+"/ckpts" # directory for model checkpoints
CSVROOT = DATAROOT+"/csvs" # directory for prediction outputs
SUBFN = DATAROOT+"/sub.csv" # final submission file path
LOGDIR = DATAROOT+"/logs" # pytorch_forecasting requirs logger

In [87]:
def seed_all(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [88]:
# prepare data features
def __date_prep(df):

    df['datetime'] = pd.to_datetime(df['datetime'])
    df['hour'] = df['datetime'].dt.hour
    df['dow'] = df['datetime'].dt.weekday
    df['date'] = df['datetime'].dt.date.astype('str')
    df['day'] = df['datetime'].dt.day
    df['month'] = df['datetime'].dt.month

    # FEATURE: saturday, sunday and speical holidays flagged as `holiday` flag
    special_days = ['2020-06-06', '2020-08-15', '2020-08-17']
    df['holiday'] = df['dow'].isin([5,6]).astype(int)
    df.loc[df.date.isin(special_days), 'holiday'] = 1

    # FEATURE: `hot` flag when the next day is holiday
    hot = df.groupby('date').first()['holiday'].shift(-1).fillna(0).astype(int)
    hot = hot.to_frame().reset_index().rename({'holiday': "hot"}, axis=1)
    df = df.merge(hot, on='date', how='left')

    # FEATURE: `cumhol` - how many days left in 연휴
    h = (df.groupby('date').first()['holiday'] != 0).iloc[::-1]
    df1 = h.cumsum() - h.cumsum().where(~h).ffill().fillna(0).astype(int).iloc[::-1]
    df1 = df1.to_frame().reset_index().rename({'holiday': "cumhol"}, axis=1)
    df = df.merge(df1, on='date', how='left')

    return df

In [89]:
# read data, process date and assign cluster number
def __read_df():
    train_columns = ['num','datetime','target','temperature','windspeed','humidity','precipitation','insolation','nelec_cool_flag','solar_flag']
    test_columns = [c for c in train_columns if c != 'target']

    train_df = pd.read_csv(DATAROOT+'/train.csv', skiprows=[0], names=train_columns)
    test_df = pd.read_csv(DATAROOT+'/test.csv', skiprows=[0], names=test_columns)

    __sz = train_df.shape[0]

    df = pd.concat([train_df, test_df])

    # assing cluster number to building
    for k, nums in CLUSTER.items():
        df.loc[df.num.isin(nums), 'cluster'] = k

    df = __date_prep(df)

    return df.iloc[:__sz].copy(), df.iloc[__sz:].copy()

In [90]:
# add aggregate(mean) target feature for 'cluster', 'building', 'mgrp' per date
def add_feats(df):
    df.reset_index(drop=True, inplace=True)

    cols = ['target']
    stats = ['mean']

    # target null in test set to null for other columns care must be taken
    g = df.groupby(['date', 'cluster'])
    for s in stats:
        col_mapper = {c:f"{s}_{c}_cluster" for c in cols}
        tr = g[cols].transform(s).rename(col_mapper, axis=1)
        df = pd.concat([df, tr], axis=1)

    g = df.groupby(['date', 'num'])
    for s in stats:
        col_mapper = {c:f"{s}_{c}_num" for c in cols}
        tr = g[cols].transform(s).rename(col_mapper, axis=1)
        df = pd.concat([df, tr], axis=1)

    g = df.groupby(['date', 'mgrp'])
    for s in stats:
        col_mapper = {c:f"{s}_{c}_mgrp" for c in cols}
        tr = g[cols].transform(s).rename(col_mapper, axis=1)
        df = pd.concat([df, tr], axis=1)

    g = df.groupby(['date'])
    for s in stats:
        col_mapper = {c:f"{s}_{c}" for c in cols}
        tr = g[cols].transform(s).rename(col_mapper, axis=1)
        df = pd.concat([df, tr], axis=1)

    return df

In [91]:
# interpolate NA values in test dataset
def interpolate_(test_df):
    # https://dacon.io/competitions/official/235736/codeshare/2844?page=1&dtype=recent
    # 에서 제안된 방법으로
    __methods = {
        'temperature': 'quadratic',
        'windspeed':'linear',
        'humidity':'quadratic',
        'precipitation':'linear',
        'insolation': 'pad'
    }

    for col, method in __methods.items():
        test_df[col] = test_df[col].interpolate(method=method)
        if method == 'quadratic':
            test_df[col] = test_df[col].interpolate(method='linear')

In [92]:
# prepare train and test data
def prep():

    train_df, test_df = __read_df()

    # get nelec_cool_flag and solar_flag from training data
    test_df = test_df.drop(['nelec_cool_flag','solar_flag'], axis=1)
    test_df = test_df.merge(train_df.groupby("num").first()[['nelec_cool_flag','solar_flag']].reset_index(), on="num", how="left")

    # interpolate na in test_df for temperature, windspeed, humidity, precipitation & insolation
    interpolate_(test_df)

    # FEATURE(mgrp): group buildings having same temperature and windspeed measurements
    s = train_df[train_df.datetime=='2020-06-01 00:00:00'].groupby(['temperature', 'windspeed']).ngroup()
    s.name = 'mgrp'
    mgrps = train_df[['num']].join(s, how='inner')

    sz = train_df.shape[0]

    df = pd.concat([train_df, test_df])
    df = df.merge(mgrps, on='num', how='left')

    # add aggregate target features
    df = add_feats(df)

    # add log target
    df["log_target"] = np.log(df.target + 1e-8)

    for col in CATE_COLS:
        df[col] = df[col].astype(str).astype('category')

    # add time index feature
    __ix = df.columns.get_loc('datetime')
    df['time_idx'] = (df.loc[:, 'datetime'] - df.iloc[0, __ix]).astype('timedelta64[h]').astype('int')

    train_df = df.iloc[:sz].copy()
    test_df = df.iloc[sz:].copy()

    return train_df, test_df

In [93]:
[os.makedirs(p, exist_ok=True) for p in (CKPTROOT, CSVROOT, LOGDIR)]

[None, None, None]

In [94]:
train_df, test_df = prep()

In [95]:
train_df

,num,datetime,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool_flag,solar_flag,...,holiday,hot,cumhol,mgrp,mean_target_cluster,mean_target_num,mean_target_mgrp,mean_target,log_target,time_idx
0,1,2020-06-01 00:00:00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,9.009332,0
1,1,2020-06-01 01:00:00,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,9.004010,1
2,1,2020-06-01 02:00:00,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,9.000499,2
3,1,2020-06-01 03:00:00,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,8.993279,3
4,1,2020-06-01 04:00:00,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,...,0,0,0,2,3223.024875,8049.78,2564.525487,2058.852819,8.992635,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19:00:00,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0,...,0,0,0,1,2431.922874,3674.16,2657.052191,2680.261446,8.322241,2035
122396,60,2020-08-24 20:00:00,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0,...,0,0,0,1,2431.922874,3674.16,2657.052191,2680.261446,8.287955,2036
122397,60,2020-08-24 21:00:00,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0,...,0,0,0,1,2431.922874,3674.16,2657.052191,2680.261446,8.180939,2037
122398,60,2020-08-24 22:00:00,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0,...,0,0,0,1,2431.922874,3674.16,2657.052191,2680.261446,8.101430,2038


In [96]:
test_df

,num,datetime,target,temperature,windspeed,humidity,precipitation,insolation,nelec_cool_flag,solar_flag,...,holiday,hot,cumhol,mgrp,mean_target_cluster,mean_target_num,mean_target_mgrp,mean_target,log_target,time_idx
122400,1,2020-08-25 00:00:00,NaN,27.800000,1.500000,74.000000,0.0,0.0,0.0,0.0,...,0,0,0,2,NaN,NaN,NaN,NaN,NaN,2040
122401,1,2020-08-25 01:00:00,NaN,27.806027,1.366667,75.017358,0.0,0.0,0.0,0.0,...,0,0,0,2,NaN,NaN,NaN,NaN,NaN,2041
122402,1,2020-08-25 02:00:00,NaN,27.639360,1.233333,76.350691,0.0,0.0,0.0,0.0,...,0,0,0,2,NaN,NaN,NaN,NaN,NaN,2042
122403,1,2020-08-25 03:00:00,NaN,27.300000,1.100000,78.000000,0.0,0.0,0.0,0.0,...,0,0,0,2,NaN,NaN,NaN,NaN,NaN,2043
122404,1,2020-08-25 04:00:00,NaN,26.787947,1.166667,79.965285,0.0,0.0,0.0,0.0,...,0,0,0,2,NaN,NaN,NaN,NaN,NaN,2044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132475,60,2020-08-31 19:00:00,NaN,28.679729,3.566667,65.645299,0.0,0.8,1.0,1.0,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,2203
132476,60,2020-08-31 20:00:00,NaN,28.313063,3.833333,66.645299,0.0,0.8,1.0,1.0,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,2204
132477,60,2020-08-31 21:00:00,NaN,27.900000,4.100000,68.000000,0.0,0.0,1.0,1.0,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,2205
132478,60,2020-08-31 22:00:00,NaN,27.900000,4.100000,68.000000,0.0,0.0,1.0,1.0,...,0,0,0,1,NaN,NaN,NaN,NaN,NaN,2206


In [97]:
from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    GroupNormalizer
) 

In [98]:
def load_dataset(train_df, validate=False):
    max_encoder_length = 24 * 7 *ENCODER_LENGTH_IN_WEEKS #5
    max_prediction_length = 24 * 7
    training_cutoff = train_df['time_idx'].max()-max_prediction_length #2040 - 24*7 = 1871

    tr_ds = TimeSeriesDataSet(
      train_df[lambda x: x.time_idx <=training_cutoff] if validate else train_df, 
      time_idx = "time_idx",
      target = "target",
      group_ids=["num"],
      min_encoder_length = 1,
      max_encoder_length = max_encoder_length, 
      min_prediction_length=1, 
      max_prediction_length=max_prediction_length,

      #Known Inputs 알고 있는 변수
      time_varying_known_categoricals = CATE_COLS, 
      time_varying_known_reals=[
            "time_idx",
            'hour',
            "temperature",
            "windspeed",
            "humidity",
            "precipitation",
            "insolation",
            'cumhol'
        ],
      target_normalizer=GroupNormalizer(groups=["num"], transformation="softplus"),
      
      #모르고 있는 변수
      time_varying_unknown_categoricals=[],
      time_varying_unknown_reals=[
            "target",
            "log_target",
            "mean_target",
            "mean_target_num",
            "mean_target_mgrp",
            "mean_target_cluster"
        ],

        
        add_relative_time_idx=True,  # add as feature
        add_target_scales=True,  # add as feature
        add_encoder_length=True,  # add as feature
    )
  

    va_ds = None
    if validate:
        va_ds = TimeSeriesDataSet.from_dataset(
        tr_ds, train_df, predict=True, stop_randomization=True
    )

    return tr_ds, va_ds

In [99]:
tr_ds, va_ds = load_dataset(train_df, validate=False)

In [100]:
# training
def fit(seed, tr_ds, va_loader=None):
    seed_all(seed) # doesn't really work as training is non-deterministic

    # create dataloaders for model
    tr_loader = tr_ds.to_dataloader(
        train=True, batch_size=BATCH_SIZE, num_workers=12
    )

    if va_loader is not None:
        # stop training, when loss metric does not improve on validation set
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",
            min_delta=1e-4,
            patience=20,
            verbose=True,
            mode="min"
        )
        lr_logger = LearningRateMonitor(logging_interval="epoch")  # log the learning rate
        callbacks = [lr_logger, early_stopping_callback]
    else:
        # gather 10 checkpoints with best traing loss
        checkpoint_callback = ModelCheckpoint(
            monitor='train_loss',
            dirpath=CKPTROOT,
            filename=f'seed={seed}'+'-{epoch:03d}-{train_loss:.2f}',
            save_top_k=10
        )
        callbacks = [checkpoint_callback]

    # create trainer
    trainer = pl.Trainer(
        max_epochs=66,
        gpus=[2],
        gradient_clip_val=PARAMS['gradient_clip_val'],
        limit_train_batches=30,
        callbacks=callbacks,
        logger=TensorBoardLogger(LOGDIR)
    )

    # use pre-deterined leraning rate schedule for final submission
    learning_rate = LRS if va_loader is None else PARAMS['learning_rate']

    # initialise model with pre-determined hyperparameters
    tft = TemporalFusionTransformer.from_dataset(
        tr_ds,
        learning_rate=learning_rate,
        hidden_size=PARAMS['hidden_size'],
        attention_head_size=PARAMS['attention_head_size'],
        dropout=PARAMS['dropout'],
        hidden_continuous_size=PARAMS['hidden_continuous_size'],
        output_size=1,
        loss=SMAPE(), # SMAPE loss
        log_interval=10,  # log example every 10 batches
        logging_metrics=[SMAPE()],
        reduce_on_plateau_patience=4,  # reduce learning automatically
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

    kwargs = {'train_dataloader': tr_loader}
    if va_loader:
        kwargs['val_dataloaders'] = va_loader

    # fit network
    trainer.fit(
        tft,
        **kwargs
    )

    best_model_path = trainer.checkpoint_callback.best_model_path
    print(f'best_model_path={best_model_path}')
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

    return best_tft

In [ ]:
seed=list(range(42, 42+10))
for s in seed:
    fit(s, tr_ds)

/home/dacon/anaconda3/envs/pch/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:360: UserWarning: Checkpoint directory /home/dacon/Dacon/junyoung/energy/Private_2위/data/ckpts exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/home/dacon/anaconda3/envs/pch/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:101: UserWarning: you defined a validation_step but have no val_dataloader. Skipping val loop
  rank_zero_warn(f'you defined a {step_name} but have no {loader_name}. Skipping {stage} loop')
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metr

Number of parameters in network: 3037.3k


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

In [ ]:
# predict 1 week
def forecast(ckpt, train_df, test_df):
    # load model
    best_tft = TemporalFusionTransformer.load_from_checkpoint(ckpt)
    max_encoder_length = best_tft.dataset_parameters['max_encoder_length']
    max_prediction_length = best_tft.dataset_parameters['max_prediction_length']

    assert max_encoder_length == 5*24*7 and max_prediction_length == 1*24*7

    # use 5 weeks of training data at the end
    encoder_data = train_df[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]

    # get last entry from training data
    last_data = train_df.iloc[[-1]]

    # fill NA target value in test data with last values from the train dataset
    target_cols = [c for c in test_df.columns if 'target' in c]
    for c in target_cols:
        test_df.loc[:, c] = last_data[c].item()

    decoder_data = test_df

    # combine encoder and decoder data. decoder data is to be predicted
    new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
    new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

    # num_labels: mapping from 'num' categorical feature to index in new_raw_predictions['prediction']
    #             {'5': 4, '6': 6, ...}
    # new_raw_predictions['prediction'].shape = (60, 168, 1)
    num_labels = best_tft.dataset_parameters['categorical_encoders']['num'].classes_

    preds = new_raw_predictions['prediction'].squeeze()

    sub_df = pd.read_csv(DATAROOT+"/sample_submission.csv")

    # get prediction for each building (num)
    for n, ix in num_labels.items():
        sub_df.loc[sub_df.num_date_time.str.startswith(f"{n} "), 'answer'] = preds[ix].numpy()

    # save predction to a csv file
    outfn = CSVROOT+'/'+(Path(ckpt).stem + '.csv')
    print(outfn)
    sub_df.to_csv(outfn, index=False)

In [ ]:
def ensemble(outfn):
    # get all prediction csv files
    fns = list(glob.glob(CSVROOT+"/*.csv"))
    df0 = pd.read_csv(fns[0])
    df = pd.concat([df0] + [pd.read_csv(fn).loc[:,'answer'] for fn in fns[1:]], axis=1)
    # get median of all predcitions
    df['median'] = df.iloc[:,1:].median(axis=1)
    df = df[['num_date_time', 'median']]
    df = df.rename({'median': 'answer'}, axis=1)
    # save to submission file
    df.to_csv(outfn, index=False)

# not used for final submission
def validate(seed, tr_ds, va_ds):
    va_loader = va_ds.to_dataloader(
        train=False, batch_size=BATCH_SIZE*10, num_workers=12
    )
    best_tft = fit(seed, tr_ds, va_loader)
    actuals = torch.cat([y[0] for x, y in iter(va_loader)])
    predictions = best_tft.predict(va_loader)
    smape_per_num = SMAPE(reduction="none")(predictions, actuals).mean(1)
    print(smape_per_num)
    print(smape_per_num.mean())

In [ ]:
import glob
print("### FORECAST ###")
for p in glob.glob(CKPTROOT + "/*.ckpt"):
    forecast(p, train_df, test_df)

In [ ]:
print("### ENSEMBLING ###")
ensemble(SUBFN)